In [ ]:
import datetime
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import json
import sqlite3

def makeUrlByDate(search, start_date, end_date):
    urls = []
    delta = end_date - start_date
    for i in range(delta.days + 1):
        date = start_date + datetime.timedelta(days=i)
        date_str = date.strftime("%Y%m%d")
        url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=" + date_str + "&de=" + date_str + "&docid=&nso=so%3Ar%2Cp%3Afrom" + date_str + "to" + date_str + "%2Ca%3A&mynews=0&refresh_start=0&related=0"
        urls.append(url)
    return urls

# 검색할 키워드 입력
search = input("검색할 키워드를 입력하세요: ")

# 크롤링할 시작 날짜 입력
start_date_str = input("\n크롤링할 시작 날짜를 입력하세요 (YYYY-MM-DD 형식): ")
start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")

# 크롤링할 종료 날짜 입력
end_date_str = input("\n크롤링할 종료 날짜를 입력하세요 (YYYY-MM-DD 형식): ")
end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")

# naver url 생성
urls = makeUrlByDate(search, start_date, end_date)

# 뉴스 크롤러 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []
for i in urls:
    url = articles_crawler(i)
    news_url.append(url)

# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", len(final_urls), '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

# 데이터 프레임으로 만들기
import pandas as pd

# 데이터 프레임 만들기
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(news_df))

# 데이터프레임을 JSON 형식으로 변환
news_json = news_df.to_json(orient='records', force_ascii=False)

# JSON 파일로 저장
with open('{}_{}.json'.format(search, now.strftime('%Y%m%d_%H시%M분%S초')), 'w', encoding='utf-8') as json_file:
    json_file.write(news_json)

# Connect to the SQLite database
conn = sqlite3.connect('news.db')
cursor = conn.cursor()

# Create a table to store the news data
cursor.execute('''CREATE TABLE IF NOT EXISTS news (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                title TEXT,
                link TEXT,
                content TEXT
                )''')

# Commit changes and close connection
conn.commit()

# Insert data into the database from the DataFrame
for index, row in news_df.iterrows():
    cursor.execute('''INSERT INTO news (date, title, link, content) VALUES (?, ?, ?, ?)''', (row['date'], row['title'], row['link'], row['content']))

# Commit changes and close connection
conn.commit()
conn.close()


In [3]:
!git clone https://github.com/cleamm/SemiProject.git

Cloning into 'SemiProject'...


In [ ]:
import datetime
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import json
import sqlite3

def makeUrlByDate(search, start_date, end_date):
    urls = []
    delta = end_date - start_date
    for i in range(delta.days + 1):
        date = start_date + datetime.timedelta(days=i)
        date_str = date.strftime("%Y%m%d")
        url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=" + date_str + "&de=" + date_str + "&docid=&nso=so%3Ar%2Cp%3Afrom" + date_str + "to" + date_str + "%2Ca%3A&mynews=0&refresh_start=0&related=0"
        urls.append(url)
    return urls

# 검색할 키워드 입력
search = input("검색할 키워드를 입력하세요: ")

# 크롤링할 시작 날짜 입력
start_date_str = input("\n크롤링할 시작 날짜를 입력하세요 (YYYY-MM-DD 형식): ")
start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")

# 크롤링할 종료 날짜 입력
end_date_str = input("\n크롤링할 종료 날짜를 입력하세요 (YYYY-MM-DD 형식): ")
end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")

# naver url 생성
urls = makeUrlByDate(search, start_date, end_date)

# 뉴스 크롤러 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []
for i in urls:
    url = articles_crawler(i)
    news_url.append(url)

# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", len(final_urls), '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

# 데이터 프레임으로 만들기
import pandas as pd

# 데이터 프레임 만들기
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(news_df))

# 데이터프레임을 JSON 형식으로 변환
news_json = news_df.to_json(orient='records', force_ascii=False)

# JSON 파일로 저장
with open('{}_{}.json'.format(search, now.strftime('%Y%m%d_%H시%M분%S초')), 'w', encoding='utf-8') as json_file:
    json_file.write(news_json)

# Connect to the SQLite database
conn = sqlite3.connect('news.db')
cursor = conn.cursor()

# Create a table to store the news data
cursor.execute('''CREATE TABLE IF NOT EXISTS news (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                title TEXT,
                link TEXT,
                content TEXT
                )''')

# Commit changes and close connection
conn.commit()

# Insert data into the database from the DataFrame
for index, row in news_df.iterrows():
    cursor.execute('''INSERT INTO news (date, title, link, content) VALUES (?, ?, ?, ?)''', (row['date'], row['title'], row['link'], row['content']))

# Commit changes and close connection
conn.commit()
conn.close()
